1. Import Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


2. Define Utility Functions for Loading and Preprocessing Data

In [ ]:
def load_and_preprocess_image(image_path, img_size=(32, 32)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return cv2.resize(image, img_size) / 255.0

def load_dataset(dataset_path, csv_file, img_size=(32, 32)):
    df = pd.read_csv(csv_file)
    images = []
    labels = []
    for _, row in df.iterrows():
        image_path = os.path.join(dataset_path, row['Path'])
        images.append(load_and_preprocess_image(image_path, img_size))
        labels.append(row['ClassId'])
    return np.array(images), np.array(labels)


3. Load and Split the Dataset

In [ ]:
# Update these paths according to your dataset location
TRAIN_PATH = 'path/to/Train'  # Update this
TEST_PATH = 'path/to/Test'    # Update this
TRAIN_CSV = 'path/to/Train.csv'
TEST_CSV = 'path/to/Test.csv'

train_images, train_labels = load_dataset(TRAIN_PATH, TRAIN_CSV)
test_images, test_labels = load_dataset(TEST_PATH, TEST_CSV)

train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)


4. Build the CNN Model

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(np.unique(train_labels)), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


5. Train the Model

In [ ]:
history = model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()


6. Create Adversarial Examples Using FGSM

In [ ]:
def create_adversarial_pattern(input_image, input_label, model):
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = model(input_image)
        loss = loss_object(input_label, prediction)
    gradient = tape.gradient(loss, input_image)
    signed_grad = tf.sign(gradient)
    return signed_grad

# Select a subset for adversarial attack demonstration
test_subset = test_images[:100]
test_labels_subset = test_labels[:100]

# Create adversarial images
epsilon = 0.1
perturbations = create_adversarial_pattern(test_subset, test_labels_subset, model)
adversarial_test_images = test_subset + perturbations * epsilon


7. Evaluate Model Performance on Original and Adversarial Images

In [ ]:
# Original test subset
loss, accuracy = model.evaluate(test_subset, test_labels_subset)
print("Original Model Accuracy: {:.2f}%".format(accuracy * 100))

# Adversarial test subset
loss, adversarial_accuracy = model.evaluate(adversarial_test_images, test_labels_subset)
print("Adversarial Model Accuracy: {:.2f}%".format(adversarial_accuracy * 100))

# Plotting comparison
plt.bar(['Original', 'Adversarial'], [accuracy, adversarial_accuracy])
plt.ylabel('Accuracy')
plt.title('Model Accuracy: Original vs Adversarial')
plt.show()


8. Visualize Some Original and Adversarial Images

In [ ]:
def display_images(image, description):
    plt.figure()
    plt.imshow(image[0])
    plt.title(description)
    plt.show()

for i in range(3):
    display_images(test_subset[i:i+1], "Original Image")
    display_images(adversarial_test_images[i:i+1], "Adversarial Image")


**DEFENSE**

1. Generate Adversarial Training Set

In [ ]:
# Generate adversarial images from a subset of the training set for quick computation
train_subset = train_images[:1000]  # Use a larger subset if computational resources allow
train_labels_subset = train_labels[:1000]

epsilon = 0.1
train_perturbations = create_adversarial_pattern(train_subset, train_labels_subset, model)
adversarial_train_images = train_subset + train_perturbations * epsilon

# Combine the adversarial images with the original training set
combined_train_images = np.concatenate((train_images, adversarial_train_images), axis=0)
combined_train_labels = np.concatenate((train_labels, train_labels_subset), axis=0)


2. Retrain the Model with the Adversarial Training Set

In [ ]:
model_adversarial_trained = models.clone_model(model)
model_adversarial_trained.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history_adversarial = model_adversarial_trained.fit(combined_train_images, combined_train_labels, epochs=10, validation_data=(val_images, val_labels))

plt.plot(history_adversarial.history['accuracy'], label='Training accuracy')
plt.plot(history_adversarial.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()


3. Evaluate the Model Performance After Adversarial Training

In [ ]:
# Original test subset
loss, accuracy = model_adversarial_trained.evaluate(test_subset, test_labels_subset)
print("Adversarially Trained Model Accuracy on Original Images: {:.2f}%".format(accuracy * 100))

# Adversarial test subset
loss, adversarial_accuracy = model_adversarial_trained.evaluate(adversarial_test_images, test_labels_subset)
print("Adversarially Trained Model Accuracy on Adversarial Images: {:.2f}%".format(adversarial_accuracy * 100))

# Plotting comparison
plt.bar(['Original - After Defense', 'Adversarial - After Defense'], [accuracy, adversarial_accuracy])
plt.ylabel('Accuracy')
plt.title('Model Accuracy After Adversarial Training')
plt.show()


4. Visualize the Effectiveness of the Defense Strategy

In [ ]:
# Before Adversarial Training
plt.bar(['Original - Before Defense', 'Adversarial - Before Defense'], [accuracy_before, adversarial_accuracy_before], alpha=0.6, color='red')

# After Adversarial Training
plt.bar(['Original - After Defense', 'Adversarial - After Defense'], [accuracy, adversarial_accuracy], alpha=0.6, color='green')

plt.ylabel('Accuracy')
plt.title('Model Accuracy Comparison')
plt.legend(['Before Defense', 'After Defense'], loc='lower right')
plt.show()
